In [1]:
#!/usr/bin/env python
# coding: utf-8


import os
import torch, pyro, numpy as np
torch.set_default_tensor_type(torch.cuda.FloatTensor)

import swyft
import click
from swyft.networks.normalization import OnlineNormalizationLayer


from torch import tensor
import torch.nn as nn
import torchvision.transforms.functional as TF

DEVICE = 'cuda'

from utils import *
from network import CustomTail, CustomHead, Mapping

import sys
sys.path.append('/home/eliasd/lensing/elias_utils')
from plotting import *

imkwargs = dict(extent=(-2.5, 2.5, -2.5, 2.5), origin='lower')




m = 1
nsub = 3
nsim = 10000

lr = 1e-3
factor = 1e-1
patience = 5
max_epochs = 30

In [2]:
SYSTEM_NAME = "ngc4414"
RUN = f'_M_m{m}_nsub{nsub}_nsim{nsim}'
assert os.path.exists(f'/nfs/scratch/eliasd/store{RUN}.sync')
SIM_PATH = f'/nfs/scratch/eliasd/store{RUN}.zarr' 
print('run', RUN)

run _M_m1_nsub3_nsim10000


In [3]:
# Set utilities
store = swyft.DirectoryStore(path=SIM_PATH)
print(f'Store has {len(store)} simulations')

torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
CONFIG = get_config(SYSTEM_NAME, str(nsub), str(m))
torch.set_default_tensor_type(torch.FloatTensor)

Loading existing store.
Store has 10042 simulations


In [4]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
ppd = CONFIG.ppd()['model_trace'].nodes
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
prior, uv = get_prior(CONFIG)

In [6]:
# Set up posterior
idx = 0
img_0 = store[idx][0]['image']
L1, L2 = torch.tensor(img_0.shape)
assert L1 == L2
L = L1.item()
print(f'L = {L}')

torch.set_default_tensor_type(torch.FloatTensor)
dataset = swyft.Dataset(nsim, prior, store)#, simhook = noise)
marginals = [i for i in range(L**2)]
post = swyft.Posteriors(dataset)

L = 40


KeyboardInterrupt: 

In [ ]:
# Train

save_name, save_path = get_name(RUN, lr, factor, patience)

n_m = 4

print(f'Training {save_name}!')
torch.set_default_tensor_type(torch.FloatTensor)
post = swyft.Posteriors(dataset)
post.add(marginals, device = DEVICE, head = CustomHead, tail = UNET)
post.train(marginals, max_epochs = max_epochs,
           optimizer_args = dict(lr=lr),
           scheduler_args = dict(factor = factor, patience = patience)
          )

# post.save(save_path)

# print(f'Loading {save_name}!')
# post = swyft.Posteriors.load(save_path).to(DEVICE)

print('Done!')

In [ ]:
plot_losses(post)

In [ ]:
assert 1 == 2

# Analyze

In [ ]:
n_m = 2
L = 40

grid = torch.linspace(0,(L-1)/L,L)
x, y = torch.meshgrid(grid, grid, indexing = 'xy')
ms = [torch.full((L*L,), m_i) for m_i in np.linspace(0, 1, 2*n_m+1)[1::2]]

coords = [tensor(()).view(1, -1)]
for m in ms:
    coords.append(
        torch.transpose(torch.stack((x.flatten(), y.flatten(), m)), 0, 1).reshape(1, -1)
    )

Map = Mapping(n_m, L)


In [ ]:
plots = torch.cat([Map.coord_to_map(coord.to(DEVICE)).cpu().squeeze() for coord in coords])
# plt_imshow(plots, 3, cbar = True)

In [ ]:
def ratio(w):
    r = np.zeros((L**2))
    for i, (key, value) in enumerate(w.items()):
        r[i] = value
    return r.T.reshape(L, L)

def get_ratio(post, obs0, coords):
    samples = post.sample_elias(N = 1, obs0 = (obs0), n_batch = 1, v = coords.view(1, -1))
    v = samples['v']
    w = samples['weights']
    return ratio(w)

In [ ]:
torch.set_default_tensor_type(torch.cuda.DoubleTensor)

re = list(post._ratios.values())[0]
head = re.head.eval()
tail = re.tail.eval()


In [ ]:
for i in np.random.randint(0, len(store), 3):
    obs0 = store[i][0]
    v0 = store[i][1]

    obs0['image'] = tensor(obs0['image'], device = 'cpu', dtype = torch.float32).unsqueeze(0)
    v0 = tensor(v0, device = 'cpu', dtype = torch.float32).unsqueeze(0)

    f = head(obs0).to(DEVICE, dtype = torch.float)

    ratios = np.zeros((n_m + 1, L, L))

    for i, coord in enumerate(coords):

        params = coord.to(DEVICE, dtype = torch.float)
        logratio = tail(f, params)
        ratio = np.exp(logratio.numpy()).reshape(n_m + 1, L, L)

        ratios[i] = ratio[i]

    lows = [-2.5, -2.5, 10]
    highs = [2.5, 2.5, 12]
    u0 = Map.coord_vu(v0, lows, highs)

    target = Map.coord_to_map(u0).squeeze().numpy()
    plots = np.concatenate((target, ratios))
    plt_imshow(plots, 2, cbar = True)
    print()